## HW3: Decision Tree, Random Forest, and Adaboost
In hw3, you need to implement decision tree, random forest and adaboost by using only numpy, then train your implemented model by the provided dataset and test the performance with testing data

Please note that only **NUMPY** can be used to implement your model, you will get no points by simply calling sklearn.tree.DecisionTreeClassifier

In [1]:
# !pip install sklearn

You should consider upgrading via the 'c:\python36\python.exe -m pip install --upgrade pip' command.


## Load data

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score

In [2]:
data = load_breast_cancer()
feature_names = data['feature_names']
print(feature_names)

['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']


In [3]:
x_train = pd.read_csv("x_train.csv")
y_train = pd.read_csv("y_train.csv")
x_test = pd.read_csv("x_test.csv")
y_test = pd.read_csv("y_test.csv")

## Question 1
Gini Index or Entropy is often used for measuring the “best” splitting of the data. Please compute the Entropy and Gini Index of provided data. Please use the formula from the course sludes on E3

In [4]:
def gini(sequence):
    num_of_data = len(sequence)
    counts = np.bincount(sequence)
    probs = counts[np.nonzero(counts)] / num_of_data
    return 1 - np.sum(np.square(probs))


def entropy(sequence):
    num_of_data = len(sequence)
    counts = np.bincount(sequence)
    probs = counts[np.nonzero(counts)] / num_of_data
    num_of_classes = len(probs)
    if(num_of_classes <= 1):
        return 0
    return - np.sum(probs * np.log2(probs))

In [5]:
# 1 = class 1,
# 2 = class 2
data = np.array([1,2,1,1,1,1,2,2,1,1,2])

In [6]:
print("Gini of data is ", gini(data))

Gini of data is  0.4628099173553719


In [7]:
print("Entropy of data is ", entropy(data))

Entropy of data is  0.9456603046006401


## Question 2
Implement the Decision Tree algorithm (CART, Classification and Regression Trees) and trained the model by the given arguments, and print the accuracy score on the test data. You should implement two arguments for the Decision Tree algorithm
1. **Criterion**: The function to measure the quality of a split. Your model should support “gini” for the Gini impurity and “entropy” for the information gain. 
2. **Max_depth**: The maximum depth of the tree. If Max_depth=None, then nodes are expanded until all leaves are pure. Max_depth=1 equals to split data once


In [8]:
class Node:
    # split_feature & split_value are used for splitting the DecisionTree
    def __init__(self, predicted_class):
        self.predicted_class = predicted_class
        self.split_feature = 0
        self.split_value = 0
        self.left = None
        self.right = None

In [38]:
class DecisionTree():
    def __init__(self, criterion='gini', max_depth=None):
        self.max_depth = max_depth
        self.criterion = criterion

    def fit(self, X, y):
        self.num_of_classes = len(set(y))  # 2 classes(0 or 1)
        self.num_of_features = X.shape[1]
        self.tree = self.buildTree(X, y)

    def bestSplit(self, X, y):
        """
        Given data X and target labels y, find the best split for a decision tree.
        """
        best_criterion = 1.0
        best_feature = None
        best_value = -1

        for feature in X.columns:
            IDs = [*range(len(X[feature]))]
            values = zip(IDs, X[feature])
            sorted_values = sorted(values, key=lambda x: x[1])

            print('Current calculating feature:', feature)
            left_values, right_values = [], sorted_values
            for value in sorted_values:
                left_values.append(value)
                right_values = right_values[1:]
                if(len(right_values) == 0 or len(left_values) == 0):
                    continue

                # Get the ground truth(y_train) using ID
                left_gt = [y.iloc[tmp[0]][0] for tmp in left_values]  # tmp[0] is the ID
                right_gt = [y.iloc[tmp[0]][0] for tmp in right_values]

                # Calculate left & right criterion
                left_gini = gini(left_gt)
                right_gini = gini(right_gt)

                # Calculate the weighted average of criterion
                left_weight = len(left_values) / (len(left_values) + len(right_values))
                right_weight = len(right_values) / (len(left_values) + len(right_values))
                weighted_criterion = (left_weight * left_gini) + (right_weight * right_gini)

                if(weighted_criterion < best_criterion):
                    best_criterion = weighted_criterion
                    best_feature = feature
                    best_value = value
        return best_feature, best_value

    def buildTree(self, X, y, depth=0):
        # Majority vote for predecting which class that current node belongs to
        # num_samples_per_class = [np.sum(y == i) for i in range(self.num_of_classes)]
        num_samples_per_class = [np.sum(y == i) for i in range(2)]
        predicted_class = np.argmax(num_samples_per_class)
        node = Node(predicted_class=predicted_class)

        if depth < self.max_depth:
            split_feature, split_value = self.bestSplit(X, y)
#             split_feature, split_value = 'mean radius', 19
            print('split feature:', split_feature)
            print('split value:', split_value)
            if split_feature is not None:
                node.split_feature = split_feature
                node.split_value = split_value

                # For each data in X,
                # filter out whether the data belongs to left sub-tree or right sub-tree
                left_filter = X[split_feature] <= split_value
                # right_filter = X[split_feature] > split_value
                X_left = X[left_filter]
                print(X_left)
                # X_right = X[right_filter]
                
#                 X_left = X[X[split_feature] <= split_value].copy()
#                 X_right = X[X[split_feature] > split_value].copy()
#                 y_left = y[y.index.isin(X_left.index)]
#                 y_right = y[y.index.isin(X_right.index)]
                
#                 node.left = self.buildTree(X_left, y_left, depth + 1)
#                 node.right = self.buildTree(X_right, y_right, depth + 1)
        return node


In [39]:
print(x_train.shape)
print(y_train.shape)
clf = DecisionTree(max_depth=1)
clf.fit(x_test, y_test)


(426, 30)
(426, 1)
Current calculating feature: mean radius
Current calculating feature: mean texture
Current calculating feature: mean perimeter
Current calculating feature: mean area
Current calculating feature: mean smoothness
Current calculating feature: mean compactness
Current calculating feature: mean concavity
Current calculating feature: mean concave points
Current calculating feature: mean symmetry
Current calculating feature: mean fractal dimension
Current calculating feature: radius error
Current calculating feature: texture error
Current calculating feature: perimeter error
Current calculating feature: area error
Current calculating feature: smoothness error
Current calculating feature: compactness error
Current calculating feature: concavity error
Current calculating feature: concave points error
Current calculating feature: symmetry error
Current calculating feature: fractal dimension error
Current calculating feature: worst radius
Current calculating feature: worst text

ValueError: operands could not be broadcast together with shapes (143,) (2,) 

### Question 2.1
Using Criterion=‘gini’, showing the accuracy score of test data by Max_depth=3 and Max_depth=10, respectively.


In [10]:
# clf_depth3 = DecisionTree(criterion='gini', max_depth=3)
# clf_depth10 = DecisionTree(criterion='gini', max_depth=10)

TypeError: __init__() got an unexpected keyword argument 'criterion'

In [ ]:
# clf_depth3.fit(x_train, y_train)
# y_pred = clf_depth3.predict(x_test)

# from sklearn.metrics import accuracy_score
# accuracy_score(y_true, y_pred)

### Question 2.2
Using Max_depth=3, showing the accuracy score of test data by Criterion=‘gini’ and Criterion=’entropy’, respectively.


In [9]:
clf_gini = DecisionTree(criterion='gini', max_depth=3)
clf_entropy = DecisionTree(criterion='entropy', max_depth=3)

- Note: All of your accuracy scores should over 0.9
- Note: You should get the same results when re-building the model with the same arguments,  no need to prune the trees
- Hint: You can use the recursive method to build the nodes


## Question 3
Plot the [feature importance](https://sefiks.com/2020/04/06/feature-importance-in-decision-trees/) of your Decision Tree model. You can get the feature importance by counting the feature used for splitting data.

- You can simply plot the feature counts for building tree without normalize the importance

![image](https://i2.wp.com/sefiks.com/wp-content/uploads/2020/04/c45-fi-results.jpg?w=481&ssl=1)

## Question 4
implement the Random Forest algorithm by using the CART you just implemented from question 2. You should implement three arguments for the Random Forest.

1. **N_estimators**: The number of trees in the forest. 
2. **Max_features**: The number of random select features to consider when looking for the best split
3. **Bootstrap**: Whether bootstrap samples are used when building tree


In [11]:
class RandomForest():
    def __init__(self, n_estimators, max_features, boostrap=True, criterion='gini', max_depth=None):
        return None

### Question 4.1
Using Criterion=‘gini’, Max_depth=None, Max_features=sqrt(n_features), showing the accuracy score of test data by n_estimators=10 and n_estimators=100, respectively.


In [12]:
clf_10tree = RandomForest(n_estimators=10, max_features=np.sqrt(x_train.shape[1]))
clf_100tree = RandomForest(n_estimators=100, max_features=np.sqrt(x_train.shape[1]))

### Question 4.2
Using Criterion=‘gini’, Max_depth=None, N_estimators=10, showing the accuracy score of test data by Max_features=sqrt(n_features) and Max_features=n_features, respectively.


In [13]:
clf_random_features = RandomForest(n_estimators=10, max_features=np.sqrt(x_train.shape[1]))
clf_all_features = RandomForest(n_estimators=10, max_features=x_train.shape[1])

- Note: Use majority votes to get the final prediction, you may get slightly different results when re-building the random forest model

## Supplementary
If you have trouble to implement this homework, TA strongly recommend watching [this video](https://www.youtube.com/watch?v=LDRbO9a6XPU), which explains Decision Tree model clearly. But don't copy code from any resources, try to finish this homework by yourself! 